In [2]:
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from diffusers import StableDiffusionPipeline
from transformers import Trainer, TrainingArguments

# Load Parquet file into a DataFrame
df = pd.read_parquet("../data/processed_sticker_dataset.parquet")
print(df.head())


                                  combined_embedding  \
0  [0.05615041, 0.06784809, -0.03342954, 0.037553...   
1  [-0.124234326, 0.07463956, -0.011985385, 0.004...   
2  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
3  [-0.06495428, -0.04292713, 0.013164402, 0.0220...   
4  [0.027918205, 0.075559475, 0.03622711, -0.0181...   

                                          image_path  
0  ../data/tensor_images/AlexatorStickers\cartoon...  
1  ../data/tensor_images/AlexatorStickers\cartoon...  
2  ../data/tensor_images/AlexatorStickers\cartoon...  
3  ../data/tensor_images/AlexatorStickers\cartoon...  
4  ../data/tensor_images/AlexatorStickers\cartoon...  


In [3]:
class StickerDataset(Dataset):
    def __init__(self, parquet_file):
        self.data = pd.read_parquet(parquet_file)
        self.transform = transforms.Compose([
            transforms.Resize((256, 256)),
            transforms.ToTensor(),
        ])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        embedding = torch.tensor(self.data.iloc[idx]["combined_embedding"])
        image_path = self.data.iloc[idx]["image_path"]
        image = Image.open(image_path).convert("RGB")  # Lazy loading
        image = self.transform(image)
        return embedding, image

In [4]:
# ✅ 2. Load Pre-trained LDM Model on CPU
model = StableDiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-2-base")
model.to("cpu")  # Ensure it runs on CPU
model.enable_gradient_checkpointing()  # Save memory

# ✅ 3. Prepare DataLoader (Optimized for CPU)
dataset = StickerDataset("stickers.parquet")
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)  # Small batch size

# ✅ 4. Training Arguments for CPU Optimization
training_args = TrainingArguments(
    output_dir="./fine_tuned_ldm",
    per_device_train_batch_size=1,  # Small batch size
    gradient_accumulation_steps=8,  # Simulate larger batch size
    num_train_epochs=5,
    save_steps=500,
    logging_steps=100,
    fp16=False,  # Disable mixed precision (CPU-only)
)


# ✅ 5. Train the Model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset
)

trainer.train()

# ✅ 6. Save the Fine-Tuned Model
model.save_pretrained("./fine_tuned_ldm")

print("✅ Fine-tuning complete! Model saved at './fine_tuned_ldm'.")

model_index.json:   0%|          | 0.00/537 [00:00<?, ?B/s]

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

c:\Users\97798\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:651: UserWarning: Not enough free disk space to download the file. The expected file size is: 1361.60 MB. The target location C:\Users\97798\.cache\huggingface\hub\models--stabilityai--stable-diffusion-2-base\blobs only has 1162.04 MB free disk space.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/738 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.36G [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.46G [00:00<?, ?B/s]

OSError: [Errno 28] No space left on device